# 1D Wave Propagation through a Heterogeneous Domain

In this notebook, we no longer solve domains monolithically (single time step), we introduce the concept of multi time
stepping with an algorithm that calls two instances of the Simple Integrator for two individually solved subdomains

## Multi-Time Step Integration 
This module describes a multi-time step integration algorithm for two subdomains through enforcing
continuity of acceleration across the interface of subdomains

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imageio 
import sys
import os
sys.path.append('..')
from SimpleIntegrator import SimpleIntegrator
from BoundaryConditions import VelBoundaryConditions as vbc
from BoundaryConditions import AccelBoundaryConditions as abc


## Create a Multi-Time Step Class

We create a module for Integrating with Multiple Time Steps
It contains the following methods:
 - Calculation of Time Step Ratios for large $\Omega_L$ and small $\Omega_s$ subdomains
    - Current Time Step Ratio $ t_{ratio}^n = \frac{t_s - t_L}{t_{PL} - t_L}$

    - Next Time Step Ratio $ t_{ratio}^{n+1} = \frac{(t_s + \Delta t_s)- t_L}{t_{PL} - t_L}$

    - where $ t_{Pi} = t_i + \Delta t_{Ci} $
    
 - Integration Communication between both subdomains with the summation and prescription of system variables
    - Mass Summation $ \mathbf{M}_\Gamma = (\mathbf{C}^T_s \mathbf{M}_s \mathbf{C}_s) + (\mathbf{C}^T_L \mathbf{M}_L \mathbf{C}_L) $

    - Internal Force Summation $ \mathbf{f}^{int}_\Gamma = \mathbf{C}_s^T \mathbf{f}^{int}_s + \mathbf{C}_L^T \mathbf{f}^{int}_L $

    - Continuity of Acceleration $ \mathbf{\ddot{u}}_{\Gamma} = \mathbf{\ddot{u}}_{\Gamma s} = \mathbf{\ddot{u}}_{\Gamma L} $

 - Calculation of Reduction Factors Quantities
    - Large Reduction Factor $ \alpha_L = 1-\frac{(t_{PL} - t_s)}{(t_{PL}-t_L)} $

    - Small Reduction Factor $ \alpha_s = 1-\frac{(t_{Ps} - t_{PL})}{(t_{Ps}-t_s)} $

 

In [2]:
class MultiTimeStep:

    """
    Constructor for the subcycling class
    It accepts two domains a large and a small one
    They are both SimpleIntegrators, and they ratio is a non-integer number:
    LARGE     |   SMALL
    *----*----*--*--*--*--*
    """
    def __init__(self, largeDomain: SimpleIntegrator, smallDomain: SimpleIntegrator):
        
        self.large = largeDomain
        self.small = smallDomain
        self.large.mass[-1] += self.small.mass[0] 
        # Time step Ratio Computations
        self.large_tpredicted = 0.0
        self.small_tpredicted = 0.0
        self.nextTimeStepRatio = 0.0 
        self.currTimeStepRatio = 0.0
        # Pullback Values
        self.large_alpha = 0.0
        self.small_alpha = 0.0

    def calc_timestep_ratios(self):
        self.small_tpredicted = self.small.t + self.small.dt
        self.large_tpredicted = self.large.t + self.large.dt
        self.currTimeStepRatio = (self.small.t - self.large.t) / (self.large_tpredicted - self.large.t)
        self.nextTimeStepRatio = ((self.small.t + self.small.dt) - self.large.t) / (self.large_tpredicted - self.large.t)

    def integrate(self):
    
        if ((self.large.t == 0) and (self.small.t == 0)):
            self.small.assemble_internal()
            self.large.assemble_internal()
            self.large.f_int[-1] += self.small.f_int[0]
            self.calc_timestep_ratios()

        while (self.nextTimeStepRatio <= 1 or (self.currTimeStepRatio <= 1 and self.nextTimeStepRatio <= 1.000001)):
            largeForce = self.large.f_int[-1]
            largeMass = self.large.mass[-1]
            def accelCoupling(): return -largeForce / largeMass

            self.small.a_bc.indexes.append(0)
            self.small.a_bc.accelerations.append(accelCoupling)

            self.small.single_tstep_integrate()
            self.small.assemble_internal()
            self.calc_timestep_ratios()

        # Compute Pullback Values
        self.alpha_L = 1 - ((self.large_tpredicted - self.small.t)/(self.large_tpredicted - self.large.t))
        self.alpha_s = 1 - ((self.small_tpredicted - self.large_tpredicted)/(self.small_tpredicted - self.small.t))

        if (self.alpha_L >= self.alpha_s):
            self.large.dt = self.alpha_L * self.large.dt
        elif (self.alpha_s > self.alpha_L):
            self.small.dt = self.alpha_s * self.small.dt

        self.large.single_tstep_integrate()
        self.large.assemble_internal()        
        self.calc_timestep_ratios()

        self.large.f_int[-1] += self.small.f_int[0]

## Output Plots for Multi Time Step Class

A Class is created to plot the FEM 1D Problems for the Heterogeneous Domain Case
This includes the creation of a plot for:
 - Velocity throughout the domain (.gif)

In [3]:
class Visualise_MultiTimestep:

    def __init__(self, upd_fullDomain: MultiTimeStep):

        self.updated = upd_fullDomain
        self.filenames = []

    def plot(self):
        self.filenames.append(f'FEM1D{self.updated.large.n}.png')
        plt.style.use('ggplot')
        plt.plot(self.updated.large.position, self.updated.large.v, "--")
        plt.plot(self.updated.small.position + self.updated.large.L , self.updated.small.v, "--")
        plt.title(f"1D Wave Propagation through Heterogeneous Media", fontsize=9)
        plt.xlabel("Domain Position (mm)", fontsize=9)
        plt.ylabel("Velocity (mm/ms)", fontsize=9)
        plt.legend([f"Updated Large Time Step Domain", "Updated Small Time Step Domain"])
        plt.savefig(f'FEM1D{self.updated.large.n}.png')
        plt.close()

    def create_gif(self):
        with imageio.get_writer('Updated_Multi-time-step.gif', mode='I') as writer:
            for filename in self.filenames:
                image = imageio.imread(filename)
                writer.append_data(image)

        for filename in set(self.filenames):
            os.remove(filename)

## 1D Heterogeneous Domain Example Domain

In this numerical example we simulate the propagation of a half sinusoidal wave through a
Heterogeneous elastic domain. Due to the difference in dilatational wave speeds, one notices that
the wave travels through at different speeds with reflection as well at the interface

Suppose a domain $\Omega$ is split into two subdomains $\Omega_L$ and $\Omega_s$ of similar discretisations, with simple elastic isotropic properties of $E_L= 207 GPa$, $E_s= 1000 GPa$ and $\rho_L=\rho_s = 7.83\times 10^{-6} kgmm^{-3}$. We state the extension of the wave equation in one dimension for the case of a stress pulse wave propagating longitudinally through a bar. As such, both subdomains $\Omega_i$ look to solve:

$
\frac{\partial \sigma_i}{\partial x} - \rho_i \frac{\partial^2u_i}{\partial t^2} = 0; \quad \text{where} \quad   u(x,t) =
    \begin{cases}
      u_L(x,t), x<125\\
      u_s(x,t), x\geq125
    \end{cases}
$

A prescribed velocity boundary condition is applied to the large domain at $x=0$ with $\dot{u}(t) = 0.01 sin (2\pi \omega_L t)$ where we define a half sine period with a frequency of $\omega_L = ({125((7.83\times 10^{-6})/207)})^{-1}$. 

In [4]:
def newCoupling():
    # Utilise same element size, drive time step ratio with Co.
    nElemLarge = 250 
    E_L = 207
    E_s = 1000
    rho = 7.83e-6
    Courant = 0.9
    Length = 125
    propTime = 1.75 * Length * np.sqrt(rho / E_L)    
    def vel(t): return vbc.velbc(t, 2 * Length , E_L, rho)
    accelBoundaryCondtions = abc(list(),list())
    upd_largeDomain = SimpleIntegrator("updated", E_L, rho, Length, 1, nElemLarge, propTime, vbc([0], [vel]), None, Co=Courant)
    upd_smallDomain = SimpleIntegrator("updated", E_s, rho, Length * 2, 1, nElemLarge, propTime, None, accelBoundaryCondtions, Co=Courant)
    upd_fullDomain = MultiTimeStep(upd_largeDomain, upd_smallDomain)
    plotfullDomain = Visualise_MultiTimestep(upd_fullDomain)
    # Solve Loop
    while(upd_fullDomain.large.t <= upd_fullDomain.large.tfinal):
        upd_fullDomain.integrate()
        plotfullDomain.plot()
    plotfullDomain.create_gif()

if __name__ == "__main__":
    newCoupling()